In [2]:
import pandas as pd
import os

# Load Excel
excel_path = r"D:\UK\00. 2024 QMUL\00. Course\Project\00. ViolenceDetectionProject\annotations\Final"
excel_file = os.path.join(excel_path, "preprocessed_dataset_1_unimodal.xlsx")

excel_data = pd.read_excel(excel_file)
df = pd.DataFrame(excel_data)
df

# Transfer End time from 'add' to nearest above 'leave'
used_add_indices = []
for i, row in df.iterrows():
    if row['Criteria'] == 'add':
        endtime = row['End time(s)']
        endframe = row['End frame']
        
        for j in range(i - 1, -1, -1):
            if df.at[j, 'Criteria'] == 'leave':
                df.at[j, 'End time(s)'] = endtime
                df.at[j, 'End frame'] = endframe
                used_add_indices.append(i)
                break

# Drop used 'add' rows
df = df.drop(index=used_add_indices).reset_index(drop=True)

df['No. Frames'] = df['End frame'] - df['Start frame'] + 1
df['No. Frames'] = df['No. Frames'].astype(int)
df['Segment Number'] = df.groupby('Video ID').cumcount() + 1
df['Segment ID'] = df['Video ID'].astype(str) + '_' + df['Segment Number'].astype(str)
df = df.drop(columns=['Segment Number'])
keep_columns = ['Video ID', 'Segment ID', 'Start time(s)', 'End time(s)', 'Start frame', 'End frame', 'No. Frames', 'Violence label(video)']
non_columns = df.columns.difference(keep_columns)
df.drop(columns = non_columns, inplace=True)

df.to_csv(os.path.join(excel_path, "Unimodal dataset_0801_ver.1.csv"), index=False)


In [49]:
average_frames_per_label = df.groupby('Violence label(video)')['No. Frames'].mean().reset_index()
average_frames_per_label['Label Name'] = average_frames_per_label['Violence label(video)'].map({0: 'non-violence', 1: 'violence'})
for _, row in average_frames_per_label.iterrows():
    print(f"{row['Label Name']}: average frames = {row['No. Frames']:.2f}")

non-violence: average frames = 904.35
violence: average frames = 324.71
